In [85]:
import requests
import json
import random
import re
import os


_baseUrl = 'https://pscp.tv/'
broadCastURL = 'https://proxsee.pscp.tv/api/v2/accessVideoPublic?broadcast_id=%s'

_apiUrl = 'https://api.periscope.tv/api/v2'

_userAgent = f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.{random.randint(0, 9999)} Safari/537.{random.randint(0, 99)}'
_apiHeaders = {
    'User-Agent': _userAgent,
    'Referer': _baseUrl,
}


def _getApiMethod(method):
    return _apiUrl + '/' + method

def _get_api_data(endpoint, params = {}):
    response = requests.get(endpoint, params = params, headers = _apiHeaders)
    obj = {}
    try:
        obj = response.json()
    except json.JSONDecodeError as e:
        print(e)
    return obj         

def _crawler_ts(b_id,live_list):
    url = _getApiMethod('accessVideoPublic')+ '?broadcast_id=' + b_id

    a = _get_api_data(url)

    if 'hls_url' in a:
        basename = os.path.basename(a['hls_url'])
        tsurl = a['hls_url'].split(basename)        
        m3u8State = True        
        while m3u8State:
            response = requests.get(a['hls_url'])
            if response.status_code == 200:        
                file = response.text            
                m3u8_lines = file.split( '\n' )
                for line in m3u8_lines:
                    b = _get_api_data(url)
                    if 'replay_url' in b:
                        m3u8State = False
                        print(b['replay_url'])
                        break
                    elif re.search("chunk_", line):
                        if line not in live_list.keys():
                            live_list[line] = tsurl[0]+line                     

            else:
                print(live_list)
                
    elif 'replay_url' in a:
        print(a['replay_url'])               

In [87]:
live_list = {}
_crawler_ts('1gqxvoAwDppKB',live_list)

https://prod-fastly-eu-central-1.video.periscope.tv/Transcoding/v1/hls/Gw2xDHPwW4EG31o4fwQVMgpCE15P7ah5nVwP9edeNUh7A1fR0M1kuFZMWvmogSPnPgXbhGrDi1xmoYLeeR-0mg/non_transcode/eu-central-1/periscope-replay-direct-prod-eu-central-1-public/playlist_16837308368623412103.m3u8?type=replay


In [63]:
print(live_list)

{'chunk_1609432783243917556_857_a.ts?type=live': 'https://prod-fastly-eu-west-1.video.periscope.tv/Transcoding/v1/hls/rsLWRCGQUbqoUlkAc_hJHgpljADI4MKoKBkTgaKq_h5QtQDvF0FabFVe-xlFTRXQ1S6I1LnTMUamPAdDE_6_cQ/non_transcode/eu-west-1/periscope-replay-direct-prod-eu-west-1-public/chunk_1609432783243917556_857_a.ts?type=live', 'chunk_1609432785169317536_858_a.ts?type=live': 'https://prod-fastly-eu-west-1.video.periscope.tv/Transcoding/v1/hls/rsLWRCGQUbqoUlkAc_hJHgpljADI4MKoKBkTgaKq_h5QtQDvF0FabFVe-xlFTRXQ1S6I1LnTMUamPAdDE_6_cQ/non_transcode/eu-west-1/periscope-replay-direct-prod-eu-west-1-public/chunk_1609432785169317536_858_a.ts?type=live', 'chunk_1609432787144972990_859_a.ts?type=live': 'https://prod-fastly-eu-west-1.video.periscope.tv/Transcoding/v1/hls/rsLWRCGQUbqoUlkAc_hJHgpljADI4MKoKBkTgaKq_h5QtQDvF0FabFVe-xlFTRXQ1S6I1LnTMUamPAdDE_6_cQ/non_transcode/eu-west-1/periscope-replay-direct-prod-eu-west-1-public/chunk_1609432787144972990_859_a.ts?type=live', 'chunk_1609432789226036040_860_a.ts?